# 多模态数据对齐

## 振动数据

In [2]:
import pandas as pd

# ===================== 1️⃣ 路径 =====================
train_csv = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/train/train_CWT.csv"
val_csv   = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/val/val_CWT.csv"
test_csv  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/test/test_CWT.csv"

# ===================== 2️⃣ 读取 =====================
train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)
test_df  = pd.read_csv(test_csv)

# ===================== 3️⃣ 基本检查 =====================
print("Train shape:", train_df.shape)
print("Val   shape:", val_df.shape)
print("Test  shape:", test_df.shape)

print("\nTrain labels:", train_df.iloc[:, -1].value_counts().sort_index())
print("\nVal labels:",   val_df.iloc[:, -1].value_counts().sort_index())
print("\nTest labels:",  test_df.iloc[:, -1].value_counts().sort_index())


Train shape: (1639, 126)
Val   shape: (272, 126)
Test  shape: (819, 126)

Train labels: label
1     116
2     118
3     117
4     117
5     116
6     117
7     118
8     117
9     118
10    118
11    117
12    117
13    117
14    116
Name: count, dtype: int64

Val labels: label
1     19
2     20
3     20
4     20
5     19
6     20
7     20
8     19
9     19
10    19
11    19
12    20
13    19
14    19
Name: count, dtype: int64

Test labels: label
1     58
2     58
3     59
4     58
5     60
6     58
7     58
8     58
9     59
10    58
11    59
12    59
13    59
14    58
Name: count, dtype: int64


## 根据CWT数据对齐电流温度信号

In [3]:
import pandas as pd
from collections import defaultdict

# ===================== 1️⃣ 路径 =====================
cwt_root = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT"
ct_csv = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/0Nm_all_classes_merged_5000.csv"

splits = {
    "train": f"{cwt_root}/train/train_CWT.csv",
    "val":   f"{cwt_root}/val/val_CWT.csv",
    "test":  f"{cwt_root}/test/test_CWT.csv",
}

out_root = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/aligned_with_CWT"

# ===================== 2️⃣ 读取 current&temp 全集 =====================
ct_df = pd.read_csv(ct_csv)
ct_label_col = ct_df.columns[-1]

print("current&temp total shape:", ct_df.shape)

# 按 label 分组（保持原始顺序）
ct_groups = {
    label: df.reset_index(drop=True)
    for label, df in ct_df.groupby(ct_label_col, sort=False)
}

# 为每个 label 准备一个“游标”
ct_ptr = defaultdict(int)

# ===================== 3️⃣ 对齐函数 =====================
def align_ct_with_cwt(cwt_df):
    cwt_label_col = cwt_df.columns[-1]
    aligned_rows = []

    for lbl in cwt_df[cwt_label_col]:
        if lbl not in ct_groups:
            raise ValueError(f"❌ current&temp 中不存在 label={lbl}")

        idx = ct_ptr[lbl]
        if idx >= len(ct_groups[lbl]):
            raise ValueError(f"❌ label={lbl} 的 current&temp 样本不足")

        aligned_rows.append(ct_groups[lbl].iloc[idx])
        ct_ptr[lbl] += 1

    return pd.DataFrame(aligned_rows)

# ===================== 4️⃣ 逐 split 对齐并保存 =====================
import os
os.makedirs(out_root, exist_ok=True)

for split, cwt_path in splits.items():
    print(f"\n=== Processing {split} ===")
    cwt_df = pd.read_csv(cwt_path)

    ct_aligned = align_ct_with_cwt(cwt_df)

    # 最后强制 label 对齐检查
    assert (ct_aligned.iloc[:, -1].values == cwt_df.iloc[:, -1].values).all(), \
        f"❌ {split} label mismatch!"

    out_path = f"{out_root}/{split}_current&temp.csv"
    ct_aligned.to_csv(out_path, index=False)

    print(f"✅ Saved: {out_path}, shape={ct_aligned.shape}")


current&temp total shape: (75000, 7)

=== Processing train ===
✅ Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/aligned_with_CWT/train_current&temp.csv, shape=(1639, 7)

=== Processing val ===
✅ Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/aligned_with_CWT/val_current&temp.csv, shape=(272, 7)

=== Processing test ===
✅ Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/aligned_with_CWT/test_current&temp.csv, shape=(819, 7)


## 对齐数据自检

In [4]:
import pandas as pd
import numpy as np

# ===================== 1️⃣ 路径 =====================
cwt_root = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT"
ct_root  = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_wl_5000/aligned_with_CWT"

splits = ["train", "val", "test"]

# ===================== 2️⃣ 自检函数 =====================
def check_alignment(cwt_df, ct_df, split_name):
    print(f"\n🔍 Self-check: {split_name}")

    cwt_label = cwt_df.columns[-1]
    ct_label  = ct_df.columns[-1]

    # ---------- ① 样本数 ----------
    assert len(cwt_df) == len(ct_df), \
        f"❌ {split_name}: sample number mismatch"

    print(f"  ✔ sample count = {len(cwt_df)}")

    # ---------- ② 逐行标签一致 ----------
    same = (cwt_df[cwt_label].values == ct_df[ct_label].values)
    mismatch_idx = np.where(~same)[0]

    if len(mismatch_idx) > 0:
        print(f"❌ {split_name}: label mismatch at indices {mismatch_idx[:10]}")
        raise AssertionError("Label alignment failed")
    else:
        print("  ✔ label alignment (row-wise)")

    # ---------- ③ label 分布一致 ----------
    cwt_dist = cwt_df[cwt_label].value_counts().sort_index()
    ct_dist  = ct_df[ct_label].value_counts().sort_index()

    assert cwt_dist.equals(ct_dist), \
        f"❌ {split_name}: label distribution mismatch"

    print("  ✔ label distribution")

    # ---------- ④ 顺序检查（同类索引单调） ----------
    for lbl in cwt_dist.index:
        idxs = np.where(cwt_df[cwt_label].values == lbl)[0]
        if not np.all(np.diff(idxs) > 0):
            raise AssertionError(f"❌ {split_name}: order broken inside label {lbl}")

    print("  ✔ intra-class order preserved")

    print(f"✅ {split_name} PASSED all checks")

# ===================== 3️⃣ 执行自检 =====================
for split in splits:
    cwt_path = f"{cwt_root}/{split}/{split}_CWT.csv"
    ct_path  = f"{ct_root}/{split}_current&temp.csv"

    cwt_df = pd.read_csv(cwt_path)
    ct_df  = pd.read_csv(ct_path)

    check_alignment(cwt_df, ct_df, split)



🔍 Self-check: train
  ✔ sample count = 1639
  ✔ label alignment (row-wise)
  ✔ label distribution
  ✔ intra-class order preserved
✅ train PASSED all checks

🔍 Self-check: val
  ✔ sample count = 272
  ✔ label alignment (row-wise)
  ✔ label distribution
  ✔ intra-class order preserved
✅ val PASSED all checks

🔍 Self-check: test
  ✔ sample count = 819
  ✔ label alignment (row-wise)
  ✔ label distribution
  ✔ intra-class order preserved
✅ test PASSED all checks
